In [5]:
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.losses import mse
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Input, Dense, Lambda, Conv2D, Flatten, Reshape, Conv2DTranspose
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import random

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
# Define the base directory where the dataset is located
base_dir = 'C:\\Users\\20203894\\Documents\\8p361'

# Define the model name
model_name = 'CVEA_Project_AI_version_'+'1'

# Define the file path for saving the model architecture
model_filepath = model_name + '.json'

# Define the file path for saving the model weights
weights_filepath = model_name + '_weights.hdf5'

# Batch size for training
batch_size = 32

# Image size
img_size = 96

# The number of epochs for training
nr_epochs = 15

#The split ratio for validation data
split = 0.85

# The class type (assuming binary classification)
class_type = 0

# latent dimensions for the VAE
latent_dim = 100

# Number of images displayed
n = 1000 

print('Model name is {}, class {}'.format(model_name, class_type))
print('Parameters-->\n Number of epochs: {}\n Batch size: {}\n Validation split: {}\n Latend dim: {}'.format(nr_epochs, batch_size, split, latent_dim))


Model name is CVEA_Project_AI_version_1, class 0
Parameters-->
 Number of epochs: 15
 Batch size: 32
 Validation split: 0.85
 Latend dim: 100


In [4]:
def get_pcam_generators(base_dir, train_percentage = 0.1, batch_size_gen=batch_size, class_type=class_type, split=split, img_size=img_size):
    """
    Uses the ImageDataGenerator function from the Keras API to return images in batches,
    train_gen for the training data and val_gen for the validation data.

    Args:
        base_dir (str): Base directory containing the dataset.
        train_percentage (float): Percentage of training data to be used
        train_batch_size (int): Batch size for training data. Default is 32.
        val_batch_size (int): Batch size for validation data. Default is 32.
        class_type (int): class type to be used. Default is 0.
        img_size (int): Size of the images. Default is 96.

    Returns:
        tuple: A tuple containing train_gen and val_gen, both are generators.
    """
    

    
    # Dataset parameters
    # Paths for training and validation data
    TRAIN_PATH = os.path.join(base_dir, 'train+val')
    VALID_PATH = os.path.join(base_dir, 'train+val')
    
    # Rescaling factor used to make the range of pixel values between 0 and 1.
    RESCALING_FACTOR = 1./255
    
    # Instantiate data generator for training data with custom percent
    train_datagen = ImageDataGenerator(rescale=RESCALING_FACTOR)
    
    train_data = datagen.flow_from_directory(TRAIN_PATH,
                                            target_size=(img_size, img_size),
                                            batch_size=batch_size_gen,
                                            subset='training',
                                            classes=[str(class_type)],
                                            class_mode='input')
    
    # Calculate the number of samples for training data based on % of dataset used
    num_train_samples = int(train_percent * len(train_data))
    
    # Extract desired aount of % of training data for training
    train_data_subset = next(train_data)[:num_train_samples]
    
    
    # Instantiate data generator for validation data
    val_datagen = ImageDataGenerator(rescale = RESCALING_FACTOR)
    val_data = val_datagen.flow_from_directory(VALID_PATH,
                                              target_size =(img_size, img_size),
                                              batch_size=batch_size_gen,
                                              class_mode="input",
                                              classes=[str(class_type)],
                                              shuffle = False)
    
    # Calculate number of samples for validation data based on split
    num_val_samples = abs((num_train_samples*(1-split))/split)
    
    # Extract x% of validation data for validation 
    val_data_subset = next(val_data)[:num_val_samples]
                                               
    
    # Return the generators
    return train_data_subset, val_data_subset

In [5]:
def construct_vae(train_gen, val_gen, weights_filepath, model_name, nr_epochs=1, latent_dim=latent_dim, batch_size=batch_size, img_size=img_size):
    
    # Define input shape and latent dimension
    input_shape = (img_size, img_size, 3)
    
    # Encoder network
    inputs = Input(shape=input_shape)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    x = Conv2D(32, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    shape_before_flattening = K.int_shape(x)
    x = Flatten()(x)
    z_mean = Dense(latent_dim)(x)
    z_log_var = Dense(latent_dim)(x)
    

    # Sampling function
    @tf.function
    def sampling(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim))
        return z_mean + K.exp(z_log_var / 2) * epsilon
    

    # Reparameterization trick
    z = Lambda(sampling)([z_mean, z_log_var])
    

    # Decoder network
    decoder_input = Input(K.int_shape(z)[1:])
    x = Dense(np.prod(shape_before_flattening[1:]), activation='relu')(decoder_input)
    x = Reshape(shape_before_flattening[1:])(x)
    x = Conv2DTranspose(128, (2, 2), activation='relu', padding='same', )(x)
    x = Conv2DTranspose(64, (2, 2), activation='relu', padding='same', strides=(2, 2))(x)
    x = Conv2DTranspose(32, (2, 2), activation='relu', padding='same', )(x)
    x = Conv2DTranspose(16, (2, 2), activation='relu', padding='same', )(x)
    x = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    # Define the VAE model
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
    decoder = Model(decoder_input, x, name='decoder')
    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name=model_name)

    # Define the VAE loss function
    reconstruction_loss = mse(K.flatten(inputs), K.flatten(outputs))
    reconstruction_loss *= input_shape[0] * input_shape[1] * input_shape[2]
    kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=1)
    B = 1000   
    vae_loss = K.mean(B * reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.add_metric(kl_loss, name="kl_loss")
    vae.add_metric(reconstruction_loss, name="reconstruction_loss")
    vae.compile(optimizer='adam')

    # Serialize model to JSON
    model_json = vae.to_json() # serialize model to JSON
    with open(model_name, 'w') as json_file:
        json_file.write(model_json)
    
    # Define EarlyStopping callback to stop training when the model stops improving
    early_stopping = EarlyStopping(monitor="val_loss", patience = 5,
                                  restore_best_weights=True)
    
    # Define other callbacks
    checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    tensorboard = TensorBoard(log_dir=os.path.join('logs', model_name))
    callbacks_list = [early_stopping, checkpoint, tensorboard]

    # Calculate number of steps per epoch for training and validation sets
    train_steps = train_gen.n // train_gen.batch_size
    val_steps = val_gen.n // val_gen.batch_size
    
    # Fit the VAE model
    vae.fit(train_gen, 
            steps_per_epoch=train_steps, 
            epochs=nr_epochs, 
            batch_size=batch_size, 
            validation_data=val_gen, 
            validation_steps=val_steps,
            callbacks=callbacks_list)
    
    # Returns the trained VAE model
    return vae

In [ ]:
def reconstruct_img(vae, val_gen, img_size=img_size):
    
    # Reconstruct images using the trained VAE model
    decoded_imgs = vae.predict(val_gen)
    
    
    # Display the original and reconstructed images
    plt.figure(figsize=(20, 4))
    for i in range(n):
        
        # Display the original image
        ax = plt.subplot(2, n, i + 1)
        img, label = val_gen.next()
        plt.imshow(img[0])
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # Display the reconstructed image
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(decoded_imgs[i].reshape(img_size, img_size,3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()



def generate_new_img(vae, base_dir, class_type=0, num_samples=10, delete_files=True, img_size=img_size):
    
    # Generate new images using random latent vectors
    random_latent_vectors  = np.random.random((num_samples, img_size, img_size, 3))
    decoded_imgs = vae.predict(random_latent_vectors)
    
    # Path to the directory where you want to save the images
    save_dir = base_dir + "/train_new_iamges/{}/".format(str(class_type))

    # Create the save directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # Optionally delete existing files in the directory
    for file in os.listdir(save_dir):
        if file.endswith('.jpg') and delete_files == True:
            os.remove(save_dir + file)

    # Iterate through generated images and save them
    for i in range(len(decoded_imgs)):
        img = decoded_imgs[i].reshape(img_size, img_size,3)
        
        # Generate a random name for the image
        random_name = ''.join(random.choices('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ', k=10))
        
        # Save the image with the random name and jpg extension
        plt.imsave(os.path.join(save_dir, random_name + ".jpg"), img)

    print("Images saved to:", save_dir)

In [ ]:
# Perform autoencoder for specified classes (for both classes = [0,1])
classes = [0,1]
for i in classes:
    # Get data generators for the specified class
    train_gen, val_gen = get_pcam_generators(base_dir, batch_size_gen=batch_size, class_type=i)
    
    # Construct VAE model and train it
    vae = construct_vae(train_gen, val_gen, weights_filepath, model_name, nr_epochs, latent_dim)
    
    # Reconstruct and display images
    reconstruct_img(vae, val_gen)
    
    # Generate new images and save them
    generate_new_img(vae, base_dir, class_type=i, num_samples=1000)